# Detecting Building defects using CNN

In [1]:
pwd


'/home/wsuser/work'

In [2]:
!pip install keras==2.2.4
!pip install tensorflow

     |████████████████████████████████| 312 kB 20.8 MB/s eta 0:00:01
     |████████████████████████████████| 50 kB 16.6 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 7.8 MB/s  eta 0:00:01
     |████████████████████████████████| 3.8 MB 68.5 MB/s eta 0:00:01
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.35.0
    Uninstalling grpcio-1.35.0:
      Successfully uninstalled grpcio-1.35.0
  Attempting uninstall: opt-einsum
    Found existing installation: opt-einsum 3.1.0
    Uninstalling opt-einsum-3.1.0:
      Successfully uninstalled opt-einsum-3.1.0
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 20210226132247
    Uninstalling flatbuffers-20210226132247:
      Successfully uninstalled flatbuffers-20210226132247


### Importing Neccessary Libraries

In [3]:
import numpy as np#used for numerical analysis
import tensorflow #open source used for both ML and DL for computation
from tensorflow.keras.models import Sequential #it is a plain stack of layers
from tensorflow.keras import layers #A layer consists of a tensor-in tensor-out computation function
#Dense layer is the regular deeply connected neural network layer
from tensorflow.keras.layers import Dense,Flatten
#Faltten-used fot flattening the input or change the dimension
from tensorflow.keras.layers import Conv2D,MaxPooling2D #Convolutional layer
#MaxPooling2D-for downsampling the image
from tensorflow.keras.preprocessing.image import ImageDataGenerator


### Image Data Agumentation

In [4]:
#setting parameter for Image Data agumentation to the training data
train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
#Image Data agumentation to the testing data
test_datagen=ImageDataGenerator(rescale=1./255)

### Loading our data and performing data agumentation

In [8]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_117a24395597428999b3b06b1904621b = 'https://s3.us.cloud-object-storage.appdomain.cloud'
else:
    endpoint_117a24395597428999b3b06b1904621b = 'https://s3.private.us.cloud-object-storage.appdomain.cloud'

client_117a24395597428999b3b06b1904621b = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='ILGYwIE3VJE0vxNUseY7m9rnf6HD2vUfarmbR9uVWAkY',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url=endpoint_117a24395597428999b3b06b1904621b)

streaming_body_1 = client_117a24395597428999b3b06b1904621b.get_object(Bucket='rockclassification-donotdelete-pr-vvospkhxvrlstn', Key='data_set.zip')['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [9]:
from io import BytesIO
import zipfile
unzip=zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths=unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [6]:
pwd

'/home/wsuser/work'

In [10]:
#performing data agumentation to train data
x_train = train_datagen.flow_from_directory("/home/wsuser/work/data_set/train_set",
    target_size=(64, 64),batch_size=5,color_mode='rgb',class_mode='categorical')
#performing data agumentation to test data
x_test = test_datagen.flow_from_directory("/home/wsuser/work/data_set/test_set",
    target_size=(64, 64),batch_size=5,color_mode='rgb',class_mode='categorical') 

Found 700 images belonging to 5 classes.
Found 300 images belonging to 5 classes.


In [11]:
print(x_train.class_indices)#checking the number of classes

{'blue calcite': 0, 'limestone': 1, 'marble': 2, 'olivine': 3, 'red crystal': 4}


In [12]:
print(x_test.class_indices)#checking the number of classes

{'blue calcite': 0, 'limestone': 1, 'marble': 2, 'olivine': 3, 'red crystal': 4}


In [13]:
from collections import Counter as c
c(x_train .labels)

Counter({0: 140, 1: 140, 2: 140, 3: 140, 4: 140})

### Creating the model

In [14]:
# Initializing the CNN
classifier = Sequential()

# First convolution layer and pooling
classifier.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
# Second convolution layer and pooling
classifier.add(Conv2D(32, (3, 3), activation='relu'))
# input_shape is going to be the pooled feature maps from the previous convolution layer
classifier.add(MaxPooling2D(pool_size=(2, 2)))

# Flattening the layers
classifier.add(Flatten())

# Adding a fully connected layer
classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dense(units=5, activation='softmax')) # softmax for more than 2



In [12]:
classifier.summary()#summary of our model

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 6272)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               802944    
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 6

### Compiling the model

In [15]:
# Compiling the CNN
# categorical_crossentropy for more than 2
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) 

## Fitting the model

In [47]:
classifier.fit_generator(
        generator=x_train,steps_per_epoch = len(x_train),
        epochs=50, validation_data=x_test,validation_steps = len(x_test))# No of images in test set

/opt/conda/envs/Python-3.8-main/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1839: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
140/140 [==============================] - 7s 47ms/step - loss: 1.3354 - accuracy: 0.4073 - val_loss: 0.9662 - val_accuracy: 0.5233
Epoch 2/50
140/140 [==============================] - 6s 43ms/step - loss: 0.7903 - accuracy: 0.6709 - val_loss: 0.6823 - val_accuracy: 0.7367
Epoch 3/50
140/140 [==============================] - 6s 43ms/step - loss: 0.5455 - accuracy: 0.7519 - val_loss: 0.8581 - val_accuracy: 0.6433
Epoch 4/50
140/140 [==============================] - 6s 42ms/step - loss: 0.4630 - accuracy: 0.8088 - val_loss: 0.4601 - val_accuracy: 0.8133
Epoch 5/50
140/140 [==============================] - 6s 42ms/step - loss: 0.4108 - accuracy: 0.8238 - val_loss: 0.9491 - val_accuracy: 0.6400
Epoch 6/50
140/140 [==============================] - 6s 41ms/step - loss: 0.4383 - accuracy: 0.8202 - val_loss: 0.6173 - val_accuracy: 0.8100
Epoch 7/50
140/140 [==============================] - 6s 42ms/step - loss: 0.3939 - accuracy: 0.8336 - val_loss: 0.4022 - val_accuracy: 0.8767

### Saving our model

In [16]:
# Save the model
classifier.save('rockrock.h5')

In [17]:
!tar -zcvf rock_classification_Model.tgz rockrock.h5

rockrock.h5


In [18]:
ls -1


data_set/
rock_classification_Model.tgz
rockrock.h5


In [35]:
!pip install watson-machine-learning-client --upgrade

In [19]:
from ibm_watson_machine_learning import APIClient
wml_credentials={
    "url":"https://us-south.ml.cloud.ibm.com",
    "apikey":"9Glbz3eQ6TvhEyvgRcFDOId8btFkXaGFza8PuPNjbWRe"
}

In [20]:
client=APIClient(wml_credentials)

In [21]:
def guid_from_space_name(client,space_name):
    space=client.spaces.get_details()
    
    return (next(item for item in space['resources'] if item['entity']['name']==space_name)['metadata']['id'])

In [22]:
space_uid=guid_from_space_name(client,'rock_classification')

print("space ID"+space_uid)


space IDda50a1da-e68e-4b09-ab5f-69c292ddde74


In [23]:
client .set.default_space(space_uid)

'SUCCESS'

In [8]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
scikit-learn_0.22-py3.6        154010fa-5b3b-4ac1-82af-4d5ee5abbc85  base
default_r3.6                   1b70aec3-ab34-4b87-8aa0-a4a3c8296a36  base
pytorch-onnx_1.3-py3.6         1bc6029

In [24]:
software_spec_uid=client.software_specifications.get_uid_by_name("tensorflow_2.4-py3.7")
software_spec_uid

'65e171d7-72d1-55d9-8ebb-f813d620c9bb'

In [25]:
model_details=client.repository.store_model(model='rock_classification_Model.tgz',meta_props={
    client.repository.ModelMetaNames.NAME:"CNN",
    client.repository.ModelMetaNames.TYPE:"keras_2.2.4",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid
    
})
model_id=client.repository.get_model_uid(model_details)                                       
                                           

Note: Warnings!! :  Model type keras_2.2.4 is deprecated. We recommend you use a supported model type. See Supported Frameworks https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/pm_service_supported_frameworks.html


In [26]:
model_id

'08a42521-4d02-439f-ae01-9a1d58147319'

In [39]:
client.repository.download(model_id,'rock_model.tar.gz')

File with name: 'rock_model.tar.gz' already exists.


WMLClientError: File with name: 'rock_model.tar.gz' already exists.

In [28]:
pwd

'/home/wsuser/work'

In [37]:
model_json = classifier.to_json()
with open("model-bw.json", "w") as json_file:
    json_file.write(model_json)

### Predicting our results

In [38]:
from tensorflow.keras.models import load_model
from keras.preprocessing import image
model = load_model("rock.h5") #loading the model for testing

OSError: SavedModel file does not exist at: rock.h5/{saved_model.pbtxt|saved_model.pb}

In [34]:
img = image.load_img(r"C:\Users\Jayasurya\Desktop\Rock-Classification-Using-CNN-main\Sample Inputs\limestone.jpg",
                     grayscale=False,target_size= (64,64))#loading of the image
x = image.img_to_array(img)#image to array
x = np.expand_dims(x,axis = 0)#changing the shape
pred = model.predict(x)#predicting the classes
pred

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Jayasurya\\Desktop\\Rock-Classification-Using-CNN-main\\Sample Inputs\\limestone.jpg'

In [7]:
import numpy as np
pred=np.argmax(pred,axis=1)

In [8]:
index=['blue calcite', 'limestone', 'marble', 'olivine', 'red crystal']
result=str(index[pred[0]])

result

'marble'

In [9]:
!pip install jupyterthemes

In [3]:
!jt -t  monokai

In [ ]:
!jt -t 